# Chapter 6 — Object References, Mutability, and Recycling

### Variables Are Not Boxes

#### Example 6-1. Variables `a` and `b` hold references to the same list, not copies of the list

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

#### Example 6-2. Variables are bound to objects only after the objects are created

In [2]:
class Gizmo:
    def __init__(self):
        print(f'Gizmo id: {id(self)}')


x = Gizmo()

Gizmo id: 140489524020752


In [3]:
y = Gizmo() * 10

Gizmo id: 140489524024352


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [4]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'quit',
 'x']

## Identity, Equality, and Aliases

#### Example 6-3. `charles` and `lewis` refer to the same object

In [5]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [6]:
id(charles), id(lewis)

(140489524071488, 140489524071488)

In [7]:
lewis['balance'] = 950

In [8]:
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

#### Example 6-4. `alex` and `charles` compare equal, but `alex` is not `charles`

In [9]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles

True

In [10]:
alex is not charles

True

### The Relative Immutability of Tuples

#### Example 6-5. `t1` and `t2` initially compare equal, but changing a mutable item inside tuple `t1` makes it different

In [11]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [12]:
id(t1[-1])

140489525623360

In [13]:
t1[-1].append(99)
t1

(1, 2, [30, 40, 99])

In [14]:
id(t1[-1])

140489525623360

In [15]:
t1 == t2

False

## Copies Are Shallow by Default

#### Example 6-6. Making a shallow copy of a list containing another list

In [16]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [17]:
l2 == l1

True

In [18]:
l2 is l1

False

In [19]:
l1.append(100)
l1[1].remove(55)
print('l1:', l1)

l1: [3, [44], (7, 8, 9), 100]


In [20]:
print('l2:', l2)

l2: [3, [44], (7, 8, 9)]


In [21]:
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)

l1: [3, [44, 33, 22], (7, 8, 9), 100]


In [22]:
print('l2:', l2)

l2: [3, [44, 33, 22], (7, 8, 9, 10, 11)]


### Deep and Shallow Copies of Arbitrary Objects

#### Example 6-8. [`bus.py`](bus.py): Bus picks and drops off passengers

In [23]:
class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

#### Example 6-9. Effects of using `copy` versus `deepcopy`

In [24]:
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(140489524296576, 140489455864176, 140489455861968)

In [25]:
bus1.drop('Bill')
bus2.passengers  # Bill is removed from bus2

['Alice', 'Claire', 'David']

In [26]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(140489455823424, 140489455823424, 140489524373056)

In [27]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

#### Example 6-10. Cyclic refernences: `b` refers to `a`, than then is append is `a`; `deepcopy` still manages to copy `a`

In [28]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [29]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

## Function Parameters as References

#### Example 6-11. A function may change any mutable object it receives

In [30]:
def f(a, b):
    a += b
    return a


x = 1
y = 2
f(x, y)

3

In [31]:
x, y

(1, 2)

In [32]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [33]:
a, b

([1, 2, 3, 4], [3, 4])

In [34]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [35]:
t, u

((10, 20), (30, 40))

### Mutable Types as Parameter Defaults: Bad Idea

#### Example 6-12. [`haunted_bus.py`](haunted_bus.py): A simple class to illustrate the danger of a mutable default

In [36]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

#### Example 6-13. Buses haunted by ghost passengers

In [37]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [38]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [39]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [40]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [41]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [42]:
bus2.passengers is bus3.passengers

True

In [43]:
bus1.passengers

['Bill', 'Charlie']

In [44]:
dir(HauntedBus.__init__)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [45]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [46]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

### Defensive Programming with Mutable Parameters

#### Example 6-15. [`twilight_bus.py`](twilight_bus.py): A simple class to show the perils of mutating received arguments

In [47]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

#### Example 6-14. Passengers disappear when dropped by a `TwilightBus`

In [48]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

## del and Garbage Collection

In [49]:
a = [1, 2]
b = a
del a
b

[1, 2]

In [50]:
b = [3]

#### Example 6-16. Watching the end of an object when no more references point to it

In [51]:
import weakref

s1 = {1, 2, 3}
s2 = s1


def bye():
    print('...like tears in the rain.')


ender = weakref.finalize(s1, bye)
ender.alive

True

In [52]:
del s1
ender.alive

True

In [53]:
s2 = 'spam'

...like tears in the rain.


In [54]:
ender.alive

False

## Tricks Python Plays with Immutables

#### Example 6-17. A tuple built from another is actually the same tuple

In [55]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

True

In [56]:
t3 = t1[:]
t3 is t1

True

#### Example 6-18. String literals may create shared objects

In [57]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [58]:
s1 = 'ABC'
s2 = 'ABC'
s1 is s2

True